In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

d:\anaconda3\envs\myenv\Lib\site-packages\torch\_subclasses\functional_tensor.py:258: UserWarning: Failed to initialize NumPy: DLL load failed while importing _multiarray_umath: 找不到指定的模块。 (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_numpy.cpp:84.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


In [3]:
words=open('names.txt','r').read().splitlines()

In [4]:
chars=sorted(list(set(''.join(words))))
stoi={s:i+1 for i,s in enumerate(chars)}
stoi['.']=0
itos={i:s for s,i in stoi.items()}

In [5]:
block_size = 3
def build_dataset(words):
    X,Y=[],[]
    for w in words:
        context = [0] * block_size
        for ch in w + '.' :
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            #print(''.join(itos[i] for i in context), '--->',itos[ix])
            context = context[1:] + [ix]

    X=torch.Tensor(X)
    X=X.to(torch.int64)
    Y=torch.Tensor(Y)
    Y=Y.to(torch.int64)
    print(X.shape,Y.shape)
    return X,Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr,Ytr=build_dataset(words[:n1])
Xdev,Ydev=build_dataset(words[n1:n2])
Xte,Yte=build_dataset(words[n2:])

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [6]:
n_embd=10
n_hidden=100

g=torch.Generator().manual_seed(2147483647)
C = torch.randn((27,n_embd),generator=g)
W1=torch.rand((n_embd*block_size,n_hidden),generator=g)
b1=torch.rand((n_hidden),generator=g)
W2=torch.randn((n_hidden,27),generator=g)
b2=torch.randn((27),generator=g)
parameters=[C,W1,b1,W2,b2]


In [7]:
for p in parameters:
    p.requires_grad=True

In [17]:
def split_loss(split):
    x,y={
        'train':(Xtr,Ytr),
        'val':(Xdev,Ydev),
        'test':(Xte,Yte),
    }[split]

    emb=C[x]
    h=torch.tanh(emb.view(-1,30)@W1 + b1)
    logits=h @ W2 + b2
    loss=F.cross_entropy(logits,y)
    print(split,loss.item())

split_loss('train')
split_loss('val')


train 2.6681649684906006
val 2.6617207527160645


In [23]:
g=torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):

    out = []
    context = [0] * block_size
    while True:
        emb2=C[torch.tensor([context])]
        h=torch.tanh(emb2.view(1,-1)@W1 + b1) 
        logits=h @ W2 + b2
        probs=F.softmax(logits,dim=1)
        ix=torch.multinomial(probs,num_samples=1,generator=g).item()
        context=context[:1] + [ix]
        out.append(ix)
        if ix==0:
            break
    
    print(''.join(itos[i] for i in out))


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x20 and 30x100)

In [20]:
for i in range(200):

    #ix=torch.randint(0,X.shape[0],(32,))
    emb=C[Xtr]
    h=torch.tanh(emb.view(-1,30)@W1 + b1)
    logits=h @ W2 + b2
    #counts=logits.exp()
    #probs=counts/counts.sum(1,keepdim=True)
    loss=F.cross_entropy(logits,Ytr)

    for p in parameters:
        p.grad=None
    loss.backward()

    lr=0.1
    for p in parameters:
        p.data += -lr * p.grad

    print(loss.item())

2.6332874298095703
2.6345291137695312
2.632697582244873
2.633934736251831
2.632110595703125
2.633343458175659
2.631526470184326
2.6327552795410156
2.6309444904327393
2.632169008255005
2.6303648948669434
2.6315858364105225
2.6297879219055176
2.631004810333252
2.629213333129883
2.6304266452789307
2.6286416053771973
2.6298508644104004
2.6280710697174072
2.629277229309082
2.6275033950805664
2.628706216812134
2.6269381046295166
2.6281378269195557
2.6263747215270996
2.627570867538452
2.6258134841918945
2.6270065307617188
2.6252543926239014
2.6264445781707764
2.624696969985962
2.6258838176727295
2.624140977859497
2.6253254413604736
2.6235873699188232
2.6247689723968506
2.623035192489624
2.6242141723632812
2.6224851608276367
2.623661518096924
2.621936321258545
2.623110294342041
2.621389150619507
2.622560501098633
2.6208434104919434
2.6220123767852783
2.6202988624572754
2.6214656829833984
2.619755983352661
2.6209206581115723
2.6192142963409424
2.6203773021698
2.6186742782592773
2.61983489990234